In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/GAN_for_Neural_Graph"

os.chdir(path)
os.listdir(path)

#Import Libraries

In [ ]:
import pandas as pd
import os
import numpy as np
import keras
from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras import optimizers
from keras import backend as K
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import Sequential
from keras import backend
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Add
from keras.utils import conv_utils
from keras.utils import to_categorical
from keras.engine import Layer
from keras.engine import InputSpec
from keras.datasets.fashion_mnist import load_data
from keras.constraints import Constraint
from keras.initializers import RandomNormal
from keras.optimizers import Adam, RMSprop
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from matplotlib import pyplot

# Load data

In [ ]:
table = pd.read_csv("./Dataset/left_table.csv")
table1 = table.loc[:,['Id','label_id']]
print(table1.head())

In [ ]:
table_dict = dict(zip(table1.Id, table1.label_id))
# print(table_dict)

In [ ]:
csv_files = os.listdir("./Dataset/FC_norm")
csv_files = [file for file in csv_files if file[-1]=='v']
print(len(csv_files))

X = []
y = []

In [ ]:
for file in csv_files:
    df = pd.read_csv(os.path.join("./Dataset/FC_norm",file),header = None)
    X.append(df.to_numpy())
    key = int(file.strip('.csv'))
#     print(key)
    y.append(int(table_dict[key]))

In [ ]:
X = np.array(X)
print(X.shape)
y = np.array(y)
print(y.shape)

In [ ]:
def loadDataset():
    return X,y

In [ ]:
table2 = table.loc[:,['label_id', 'Age', 'Gender']]
print(table2.head())

In [ ]:
#Normalize Age column
table2['Age'] = (table2['Age'] - table2['Age'].min()) / (table2['Age'].max() - table2['Age'].min())

#Make Gender 0, 1 instead of 1, 0
table2['Gender'] = table2['Gender'].replace([2],0)
print(table2)

In [ ]:
#One-hot encose the class labels
class_labels = np.array(list(table2['label_id']))
class_labels = np.expand_dims(class_labels, axis=1)

mlb = MultiLabelBinarizer()
class_labels = mlb.fit_transform(class_labels)
print(class_labels)

In [ ]:
encoded_data = []
for label, row in zip(class_labels, table2.itertuples()):
    label = list(label)
    age = [row.Age]
    gender = [row.Gender]
#     print(type(label), type(age), type(gender))
#     print(label, age, gender)
    encoded_data.append(label+age+gender)
    #print(label+age+gender)

# encoded_data = np.array(encoded_data)
for i in range(2):
    print(encoded_data[i])

In [ ]:
enc_idx = np.arange(0,len(encoded_data))

sample_idx = np.random.choice(enc_idx, size = 10)

print(sample_idx)

for idx in sample_idx:
    print(encoded_data[idx])

In [ ]:
sample_idx = np.random.choice(len(encoded_data), size = 10)
print(sample_idx)

#Hyper parameters

In [ ]:
# Discriminator
DECAY = 0.0005
ALPHA = 0.33
LR_D = 0.0001
BETA_D = 0.5
FE_CHANNEL = 128
MERGE_CHANNEL = 64


# Generator
D = 10
STD = 0.02

# GAN
LR_G = 0.0001
BETA_G = 0.5

# Loss weights
LOSS_WEIGHTS = [1.0, 1.0]

# Train function
LATENT_DIM = 50
BATCH_SIZE = 64

'''
The number of E2E layers
Channel size of E2E layers
Dropout
'''

#Model function

In [ ]:
# define E2E layer
from keras import backend as K
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.engine import Layer
from keras.engine import InputSpec
from keras.utils import conv_utils

class E2E_conv(Layer):
  def __init__(self, rank,
         filters,
         kernel_size,
         strides=1,
         padding='valid',
         activation=None,
         kernel_initializer='glorot_uniform',
         kernel_regularizer=None,
         kernel_constraint=None,
         **kwargs):
    super(E2E_conv, self).__init__(**kwargs)
    self.rank = rank
    self.filters = filters
    self.kernel_size = conv_utils.normalize_tuple(kernel_size, rank, 'kernel_size')
    self.strides = conv_utils.normalize_tuple(strides, rank, 'strides')
    self.padding = conv_utils.normalize_padding(padding)
    self.activation = activations.get(activation)
    self.kernel_initializer = initializers.get(kernel_initializer)
    self.kernel_regularizer = regularizers.get(kernel_regularizer)
    self.kernel_constraint = constraints.get(kernel_constraint)
    self.input_spec = InputSpec(ndim=self.rank + 2)

  def build(self, input_shape):
    channel_axis = -1
    if input_shape[channel_axis] is None:
      raise ValueError('The channel dimension of the inputs'
               'should be defined. Found `None`.')
    input_dim = input_shape[channel_axis]
    kernel_shape = self.kernel_size + (input_dim, self.filters)

    self.kernel = self.add_weight(shape=kernel_shape,
                    initializer=self.kernel_initializer,
                    name='kernel',
                    regularizer=self.kernel_regularizer,
                    constraint=self.kernel_constraint)
    
    # Set input spec.
    self.input_spec = InputSpec(ndim=self.rank + 2,
                   axes={channel_axis:input_dim})
    self.built = True

  def call(self, inputs):
    kernel_shape = K.get_value(self.kernel).shape
    d = kernel_shape[1]
    kernellxd = K.reshape(self.kernel[0,:], (1, kernel_shape[1], kernel_shape[2], kernel_shape[3]))  # row vector
    kerneldxl = K.reshape(self.kernel[1,:], (kernel_shape[1], 1, kernel_shape[2], kernel_shape[3]))  # column vector
    convlxd = K.conv2d(
        inputs,
        kernellxd,
        strides=self.strides,
        padding=self.padding)
    convdxl = K.conv2d(
        inputs,
        kerneldxl,
        strides=self.strides,
        padding=self.padding)
    concat1 = K.concatenate([convdxl]*d, axis=1)
    concat2 = K.concatenate([convlxd]*d, axis=2)
    return concat1 + concat2

  def compute_output_shape(self, input_shape):
    return (input_shape[0], input_shape[1], input_shape[2], self.filters)

  def get_config(self):
    config = {
        'rank': self.rank,
        'filters': self.filters,
        'kernel_size': self.kernel_size,
        'strides': self.strides,
        'padding': self.padding,
        'activation': activations.serialize(self.activation),
        'kernel_initializer': initializers.serialize(self.kernel_initializer),
        'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
        'kernel_constraint': constraints.serialize(self.kernel_constraint)
    }
    base_config = super(E2E_conv, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

# define D

In [ ]:
# define the standalone discriminator model
def define_discriminator(image_shape=(200,200,1), n_classes=3):
  # weight regularization
  reg = regularizers.l2(DECAY)
  # weight initialization
  kernel_init = initializers.he_uniform()
  # image input
  in_image = Input(shape=image_shape, name='in_image')

  # E2E layer
  fe = E2E_conv(2, 32, (2, 200), kernel_regularizer=reg)(in_image)  
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=ALPHA)(fe)

  fe = E2E_conv(2, 64, (2, 200), kernel_regularizer=reg)(fe)     
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=ALPHA)(fe)
  fe = Dropout(0.5)(fe)

  # E2N layer
  temp1 = Conv2D(128, (1, 200), kernel_regularizer=reg, name='row')(fe)  
  temp2 = Conv2D(128, (200, 1), kernel_regularizer=reg, name='column')(fe)
  temp2 = Reshape((200, 1, 128))(temp2)
  fe = Add()([temp1, temp2])
  fe = BatchNormalization()(fe)                          
  fe = LeakyReLU(alpha=ALPHA)(fe)
  fe = Dropout(0.5)(fe)

  # N2G layer
  fe = Conv2D(256, (200, 1), kernel_regularizer=reg)(fe) 
  fe = BatchNormalization()(fe)          
  fe = LeakyReLU(alpha=ALPHA)(fe)
  fe = Dropout(0.5)(fe)

  # flatten feature maps
  fe = Flatten()(fe)

  fe = Dense(FE_CHANNEL, kernel_regularizer=reg, kernel_initializer=kernel_init)(fe)
  fe = LeakyReLU(alpha=ALPHA)(fe)
  fe = Dropout(0.5)(fe)

  merge = Dense(MERGE_CHANNEL, kernel_regularizer=reg, kernel_initializer=kernel_init)(fe)
  merge = LeakyReLU(alpha=ALPHA)(merge)
  merge = Dropout(0.5)(merge)

  # real/fake output
  out1 = Dense(1, activation='sigmoid', name='valid')(merge)
  # class label output
  out2 = Dense(n_classes, activation='softmax', name='class')(merge)
  # define model
  model = Model(in_image, [out1, out2], name="Discriminator")
  return model

# define G

In [ ]:
# define the standalone generator model
def define_generator(latent_dim=50, n_classes=3, d=D):
  #Initailize Weights
  init = RandomNormal(stddev=STD)
    
  #Take in noise as input
  in_z = keras.Input(shape=(latent_dim,))
  print(f"Shape of Noise Vector: {in_z.shape}")
  
  #Create a dense layer
  dense = keras.layers.Dense(200*d, activation="relu", kernel_initializer = init)
  
  X = dense(in_z)
  X = keras.layers.Reshape((200,d))(X)
  print(f"Shape of X: {X.shape}")

  A = keras.layers.Dot(axes=(2, 2))([X,X])
  A = keras.backend.expand_dims(A, axis = -1)

  A = Activation('tanh')(A)
  print(f"Shape of A: {A.shape}")
  
  # define model
  model = Model(in_z, A, name="Generator")
  return model

#define GAN

In [ ]:
def all_model(latent_dim=50):
  # define D & G
  d_model = define_discriminator()
  g_model = define_generator(latent_dim)

  # compile D
  opt = optimizers.Adam(lr=LR_D, beta_1=BETA_D)
  d_model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], loss_weights=LOSS_WEIGHTS, optimizer=opt, metrics=['acc'])

  # define GAN
  d_model.trainable = False
 
  in_noise = keras.Input(shape=(latent_dim,))
  img = g_model(in_noise)

  valid, label = d_model(img)
  gan_model = Model(in_noise, [valid, label], name="GAN")

  opt = optimizers.Adam(lr=LR_G, beta_1=BETA_G)
  gan_model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], loss_weights=LOSS_WEIGHTS, optimizer=opt, metrics=['acc'])

  return d_model, g_model, gan_model

In [ ]:
d_model, g_model, gan_model = all_model(LATENT_DIM)
d_model.summary()
g_model.summary()
gan_model.summary()

#Auxiliary function

In [ ]:
# load images
def load_real_samples():
  # load dataset
  (X, y)= loadDataset()
  # expand to 3d, e.g. add channels
  X = np.expand_dims(X, axis=-1)
  # convert from ints to floats
  X = X.astype('float32')
  print(X.shape, y.shape)
  X_train, X_remain, y_train, y_remain = train_test_split(X, y, test_size=0.2, random_state=42)
  X_val, X_test, y_val, y_test = train_test_split(X_remain, y_remain, test_size=0.5, random_state=42)

  print(f"Training Data, X shape: {X_train.shape}, y shape: {y_train.shape}")
  print(f"Validation Data, X shape: {X_val.shape}, y shape: {y_val.shape}")
  print(f"Test Data, X shape: {X_test.shape}, y shape: {y_test.shape}")

  return [X_train, y_train],[X_val, y_val],[X_test, y_test]
     
# select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = np.random.randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix] 
	y = np.ones((n_samples, 1))              
	return X, [y, labels]

def generate_random_ecodings(n_samples):
  enc_idx = np.arange(0,len(encoded_data))
  sample_idx = np.random.choice(enc_idx, size = n_samples)
  samples = []
  labels = []
  #print(sample_idx)
  for idx in sample_idx:
    samples.append(encoded_data[idx][:3])
    label = encoded_data[idx][:3]
    if label[0]==1:
      labels.append(0)
    elif label[1]==1:
      labels.append(1)
    else:
      labels.append(2)
  return np.array(samples), np.array(labels)

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=3):
  #Generate noise, 3 dimensions short of latent_dim
  z_noise = np.random.normal(0, 1, size=[n_samples,latent_dim-3])   # Gaussian distribution
  #Generate encoding of 3 dimensions
  z_encoding, labels = generate_random_ecodings(n_samples)
  #Concatenate z_noise and z_encoding to create input of latent_dim
  z_input = np.concatenate((z_noise, z_encoding), axis = 1)
  return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
  # generate points in latent space
  z_input, labels_input = generate_latent_points(latent_dim, n_samples)
  # predict outputs
  images = generator.predict(z_input)
  y = np.zeros((n_samples, 1))           
  return images, [y, labels_input]

# generate samples and save as a plot and save the model
def summarize_performance(step, d_model):
  path = 'AC_Brain/mode3'
  filename1 = path + '/weights/d_model_%04d.h5' % (step+1)
  d_model.save_weights(filename1)
  print('>Saved: %s' % filename1)

# create a line plot of loss for the gan and save to file
def plot_history(train_hist, validation_hist):
  path = 'AC_Brain/mode3'
  # dr_v_loss1, df_v_loss1, g_v_loss1, dr_v_acc1, df_v_acc1, dr_c_acc1, df_c_acc1 = train_hist
  # # plot train_data loss
  # pyplot.plot(dr_v_loss1, label='D-validity-real')
  # pyplot.plot(df_v_loss1, label='D-validity-fake')
  # pyplot.plot(g_v_loss1, label='G-validity')
  # pyplot.legend()
  # pyplot.savefig(path + '/plot_train_loss.pdf')
  # pyplot.close()
 
  # plot train_datad accuracy
  # pyplot.plot(dr_v_acc1, label='validity-real')
  # pyplot.plot(df_v_acc1, label='validity-fake')
  # pyplot.legend()
  # pyplot.savefig(path + '/plot_train_valid_acc.pdf')
  # pyplot.close()

  # pyplot.plot(dr_c_acc1, label='class-real')
  # pyplot.plot(df_c_acc1, label='class-fake')
  # pyplot.legend()
  # pyplot.savefig(path + '/plot_train_class_acc.pdf')
  # pyplot.close()
 
  dr_v_loss2, df_v_loss2, dr_v_acc2, df_v_acc2, dr_c_acc2 = validation_hist
  # # plot validation_data loss
  # pyplot.plot(dr_v_loss2, label='validity-real')
  # pyplot.plot(df_v_loss2, label='validity-fake')
  # pyplot.legend()
  # pyplot.savefig(path + '/plot_validation_loss.pdf')
  # pyplot.close()

  # plot validation_data accuracy
  pyplot.plot(dr_v_acc2, label='validity-real')
  pyplot.plot(df_v_acc2, label='validity-fake')
  pyplot.plot(dr_c_acc2, label='class-real')
  pyplot.legend()
  pyplot.savefig(path + '/plot_validation_acc.pdf')
  pyplot.close()

#Training

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, val_dataset, n_epochs=300, latent_dim=LATENT_DIM, n_batch=BATCH_SIZE):
  epoch=0
  # calculate the number of batches per training epoch
  bat_per_epo = int(dataset[0].shape[0] / n_batch)
  # calculate the number of training iterations
  n_steps = bat_per_epo * n_epochs
  # calculate the real/fake batch_size
  half_batch = int(n_batch / 2)
  # prepare lists for train_data hist
  dr_v_loss1, df_v_loss1, g_v_loss1, dr_v_acc1, df_v_acc1, dr_c_acc1, df_c_acc1 = list(), list(), list(), list(), list(), list(), list()
  # prepare lists for validation_data hist
  dr_v_loss2, df_v_loss2, dr_v_acc2, df_v_acc2, dr_c_acc2 = list(), list(), list(), list(), list()

  # manually enumerate epochs
  for i in range(n_steps):
    #----------------------------------------
    # update discriminator model weights
    #----------------------------------------

    # get randomly selected 'real' samples
    X_real, [y_real, labels_real] = generate_real_samples(dataset, half_batch)
    dr_metrics = d_model.train_on_batch(X_real, [y_real, labels_real])
    # generate 'fake' 
    X_fake, [y_fake, labels_fake] = generate_fake_samples(g_model, latent_dim, half_batch)
    df_metrics = d_model.train_on_batch(X_fake, [y_fake, labels_fake])

    # summarize the loss and accuracy
    d_metrics = 0.5 * np.add(dr_metrics, df_metrics)

    #----------------------------------------
    # update the generator via the discriminator's error
    #----------------------------------------

    # prepare points in latent space as input for the generator
    [z_input, z_labels] = generate_latent_points(latent_dim, n_batch)   
    y_gan = np.ones((n_batch, 1)) 
    g_metrics = gan_model.train_on_batch(z_input, [y_gan, z_labels])

    # summarize loss on this batch
    print('STEP:%d, D{v_l: %.3f, v_acc: [%.1f| %.1f| %.1f], c_acc: [%.1f| %.1f]}  G{v_l: %.3f, v_acc: %.1f, c_acc: %.1f}'
        % (i+1, d_metrics[1], 100*dr_metrics[3], 100*df_metrics[3], 100*d_metrics[3], 100*dr_metrics[4], 100*df_metrics[4], 
          g_metrics[1], 100*g_metrics[3], 100*g_metrics[4]))
    # metrics[0]: loss, metrics[1]: validity_loss, metrics[2]: classification_loss, metrics[3]: validity_accuracy, metrics[4]: classification_accuracy

    # record history
    dr_v_loss1.append(dr_metrics[1])
    df_v_loss1.append(df_metrics[1])
    g_v_loss1.append(g_metrics[1])
    dr_v_acc1.append(dr_metrics[3])
    df_v_acc1.append(df_metrics[3])
    dr_c_acc1.append(dr_metrics[4])
    df_c_acc1.append(df_metrics[4])

    #----------------------------------------
    # evaluation
    #----------------------------------------
    if (i+1) % (bat_per_epo) == 0:
      epoch+=1
      # generate real validation data
      X_r_val, labels_r_val = val_dataset
      num_test = X_r_val.shape[0]
      # generate fake validation data
      y_r_val = ones((num_test, 1))
      X_f_val, [y_f_val, labels_f_val] = generate_fake_samples(g_model, latent_dim, num_test)

      print(f"\nValidation Metrics of Discriminator:")
      # evaluate both real and fake valid_dataset
      valid_metrics_r = d_model.evaluate(X_r_val, [y_r_val, labels_r_val], verbose=1)
      valid_metrics_f = d_model.evaluate(X_f_val, [y_f_val, labels_f_val], verbose=1)

      v_acc = 50 * (valid_metrics_r[3] + valid_metrics_f[3])  
      three_c_acc = 100 * valid_metrics_r[4]   

      # two class accuracy
      _, labels_pred = d_model.predict(X_r_val)
      labels_pred = np.argmax(labels_pred, axis=1)
      # print('val {class_0: %d, class_1: %d, class_2: %d}' % (np.sum(labels_r_val==0), np.sum(labels_r_val==1), np.sum(labels_r_val==2)))
      # print('pred {class_0: %d, class_1: %d, class_2: %d}' % (np.sum(labels_pred==0), np.sum(labels_pred==1), np.sum(labels_pred==2)))
      labels_2_val, labels_2_pred = labels_r_val.copy(), labels_pred.copy()
      # classify 2 as 1
      labels_2_val[labels_2_val==2] = 1
      labels_2_pred[labels_2_pred==2] = 1
      # calculate the accuracy 
      correct = np.sum(labels_2_val==labels_2_pred)
      two_c_acc = correct / num_test * 100
           
      print('average v_acc: %.3f, three class acc: %.3f, two class acc: %.3f' % (v_acc, three_c_acc, two_c_acc))
      print("="*100)
      # save good models
      if three_c_acc > 64 or two_c_acc > 67:
      # if three_c_acc > 65:
        summarize_performance(i, d_model)

      # record history
      dr_v_loss2.append(valid_metrics_r[1])
      df_v_loss2.append(valid_metrics_f[1])
      dr_v_acc2.append(valid_metrics_r[3])
      df_v_acc2.append(valid_metrics_f[3])
      dr_c_acc2.append(valid_metrics_r[4])

    # print epoch
    if (i+1) % (bat_per_epo * 10) == 0:
      print(f"Epoch: {epoch}")
      print("="*100)
   
      # plot history
      train_hist = [dr_v_loss1, df_v_loss1, g_v_loss1, dr_v_acc1, df_v_acc1, dr_c_acc1, df_c_acc1]
      validation_hist = [dr_v_loss2, df_v_loss2, dr_v_acc2, df_v_acc2, dr_c_acc2]
      plot_history(train_hist, validation_hist)
      point = [dr_v_acc2, df_v_acc2, dr_c_acc2]
      np.save('AC_Brain/mode3/point_3', point)

In [ ]:
# load image data
train_data, val_data, test_data = load_real_samples()

In [ ]:
epochs = 100

# define model
discriminator, generator, gan_model = all_model(LATENT_DIM)
# train model
train(generator, discriminator, gan_model, train_data, val_data, n_epochs=epochs)

In [ ]:
pathway = '/weights/d_model_0720.h5'

test(LATENT_DIM, test_data, pathway)

# Test


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics

def calculate_score(test, pred):
  new_pred = np.zeros((pred.shape[0], 2))
  new_pred[:, 0] = pred[:, 0]
  new_pred[:, 1] = pred[:, 1] + pred[: ,2]
  score = new_pred[:, 1]
  return score

def test(latent_dim, test_dataset, pathway):
  # load model
  path = 'AC_Brain/mode3'

  d_model, _, _ = all_model(LATENT_DIM)
  d_model.load_weights(path + pathway)

  X_test, labels_test = test_dataset
  num_test = X_test.shape[0]
  y_test = ones((num_test, 1))

  print(f"\nValidation Metrics of Discriminator:")
  test_metrics = d_model.evaluate(X_test, [y_test, labels_test], verbose=1)
  v_acc = 100 * test_metrics[3]
  three_c_acc = 100 * test_metrics[4]

  # two class accuracy
  _, temp_pred = d_model.predict(X_test)
  labels_pred = np.argmax(temp_pred, axis=1)

  correct = np.sum(labels_pred==labels_test)
  acc = correct / num_test * 100
  print('test: %.3f' % acc)

  labels_2_test, labels_2_pred = labels_test.copy(), labels_pred.copy()
  # classify 2 as 1
  labels_2_test[labels_2_test==2] = 1
  labels_2_pred[labels_2_pred==2] = 1
  # calculate the accuracy
  correct = np.sum(labels_2_test==labels_2_pred)
  two_c_acc = correct / num_test * 100
  print('average v_acc: %.3f, three class acc: %.3f, two class acc: %.3f' % (v_acc, three_c_acc, two_c_acc))
  print("="*100)

  # three class confusion metrics
  target_names = ['class 0', 'class 1', 'class 2']
  print('three class:')

  # calculate precision, recall, f1 score
  print(classification_report(labels_test, labels_pred, target_names=target_names))

  # calculate AUC
  onehot = to_categorical(labels_test, num_classes=3)
  AUC = metrics.roc_auc_score(onehot, temp_pred, multi_class='ovr')
  print('AUC: ', AUC)
  print("="*100)

  # two class confusion metrics
  target_names = ['class 0', 'class 1']
  print('two class:')
  # calculate precision, recall, f1 score
  print(classification_report(labels_2_test, labels_2_pred, target_names=target_names))

  # calcuate specificity
  tn, fp, fn, tp = confusion_matrix(labels_2_test, labels_2_pred).ravel()
  specificity = tn / (tn+fp)
  print('specificity:', specificity)

  # calculate AUC
  score = calculate_score(labels_2_test, temp_pred)
  AUC = metrics.roc_auc_score(labels_2_test, score)
  print('AUC: ', AUC)
  print("="*100)

In [ ]:
# load data
train_data, val_data, test_data = load_real_samples()

test(LATENT_DIM, test_data)